## Ingesting PDF

In [19]:
!ollama pull nomic-embed-text
!ollama pull mistral

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers ⠋ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏

In [20]:
%pip install pytesseract
%pip install pdf2image
%pip install --q unstructured langchain
%pip install langchain_community
%pip install unstructured_inference
%pip install pdfminer.six
%pip install pillow_heif
%pip install poppler_utils
%pip install unstructured_pytesseract
%pip install --q chromadb
%pip install --q langchain-text-splitters
%pip install --upgrade setuptools

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



In [21]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader(r"D:/JOb/AI/rag/conveng.pdf")
data = loader.load()

In [22]:
data

[Document(metadata={'source': 'D:/JOb/AI/rag/conveng.pdf', 'page': 0}, page_content='1\nUNITED NATIONS FRAMEWORK CONVENTION\nON CLIMATE CHANGE\nUNITED NATIONS\n1992'),
 Document(metadata={'source': 'D:/JOb/AI/rag/conveng.pdf', 'page': 1}, page_content="2\nUNITED NATIONS FRAMEWORK CONVENTION ON CLIMATE CHANGE\nThe Parties to this Convention ,\nAcknowledging  that change in the Earth's climate and its\nadverse effects are a common concern of humankind,\nConcerned  that human activities have been substantially\nincreasing the atmospheric concentrations of greenhouse gases,\nthat these increases enhance the natural greenhouse effect,\nand that this will result on average in an additional warming\nof the Earth's surface and atmosphere and may adversely affect\nnatural ecosystems and humankind,\nNoting that the largest share of historical and current\nglobal emissions of greenhouse gases has originated in\ndeveloped countries, that per capita emissions in developing\ncountries are still rela

In [23]:
# Preview first page
data[1].page_content

"2\nUNITED NATIONS FRAMEWORK CONVENTION ON CLIMATE CHANGE\nThe Parties to this Convention ,\nAcknowledging  that change in the Earth's climate and its\nadverse effects are a common concern of humankind,\nConcerned  that human activities have been substantially\nincreasing the atmospheric concentrations of greenhouse gases,\nthat these increases enhance the natural greenhouse effect,\nand that this will result on average in an additional warming\nof the Earth's surface and atmosphere and may adversely affect\nnatural ecosystems and humankind,\nNoting that the largest share of historical and current\nglobal emissions of greenhouse gases has originated in\ndeveloped countries, that per capita emissions in developing\ncountries are still relatively low and that the share of\nglobal emissions originating in developing countries will grow\nto meet their social and development needs,\nAware of the role and importance in terrestrial and\nmarine ecosystems of sinks and reservoirs of greenhouse 

## Vector Embeddings

In [24]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED       
mistral:latest         	f974a74358d6	4.1 GB	32 seconds ago	
nomic-embed-text:latest	0a109f422b47	274 MB	34 seconds ago	


In [25]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [26]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [27]:
chunks

[Document(metadata={'source': 'D:/JOb/AI/rag/conveng.pdf', 'page': 0}, page_content='1\nUNITED NATIONS FRAMEWORK CONVENTION\nON CLIMATE CHANGE\nUNITED NATIONS\n1992'),
 Document(metadata={'source': 'D:/JOb/AI/rag/conveng.pdf', 'page': 1}, page_content="2\nUNITED NATIONS FRAMEWORK CONVENTION ON CLIMATE CHANGE\nThe Parties to this Convention ,\nAcknowledging  that change in the Earth's climate and its\nadverse effects are a common concern of humankind,\nConcerned  that human activities have been substantially\nincreasing the atmospheric concentrations of greenhouse gases,\nthat these increases enhance the natural greenhouse effect,\nand that this will result on average in an additional warming\nof the Earth's surface and atmosphere and may adversely affect\nnatural ecosystems and humankind,\nNoting that the largest share of historical and current\nglobal emissions of greenhouse gases has originated in\ndeveloped countries, that per capita emissions in developing\ncountries are still rela

In [28]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

OllamaEmbeddings: 100%|██████████| 78/78 [03:34<00:00,  2.74s/it]


In [29]:
vector_db

## Retrieval

In [30]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [31]:
# LLM from Ollama
local_model = "mistral"
llm = ChatOllama(model=local_model)

In [32]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [33]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [34]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [35]:
chain.invoke(input(""))

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.16s/it]


" The context indicates that the parties have agreed to take several measures to prevent climate change and mitigate its adverse effects. These include:\n\n1. Implementing policies and measures that are cost-effective, comprehensive, cover all relevant sources, sinks, and reservoirs of greenhouse gases, and encompass all economic sectors.\n\n2. Facilitating adequate adaptation to climate change.\n\n3. Promoting and cooperating in the development, application, and diffusion (including transfer) of technologies, practices, and processes that control, reduce, or prevent anthropogenic emissions of greenhouse gases not controlled by the Montreal Protocol in all relevant sectors such as energy, transport, industry, agriculture, forestry, and waste management.\n\n4. Promoting sustainable management and conservation of sinks and reservoirs of greenhouse gases, where appropriate.\n\n5. Taking precautionary measures to anticipate, prevent, or minimize the causes of climate change and mitigate it

In [36]:
# Delete all collections in the db
vector_db.delete_collection()